# Preprocessing data enem 2022

In this notebook we will treat the data in rder to make it apropriate for machine learning models.

In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import pylab as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

pd.set_option('display.float_format', lambda x: '%.2f' % x)

Let's start by analising the head of our data and the information about it.

In [2]:
data_DF = pd.read_csv('data_DF.csv')
data_DF.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,Q001,Q002,Q006,Q025,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
0,210055243739,4,F,1,1,2,0,D,D,C,B,390.10,537.70,442.70,441.40,600.00
1,210055092856,2,F,1,3,2,0,D,E,E,B,564.00,581.30,604.50,590.40,780.00
2,210054977951,5,F,1,2,1,0,B,B,C,B,478.30,410.30,445.60,469.00,340.00
3,210055094382,2,F,1,1,1,1,C,D,B,B,483.00,487.30,501.40,540.00,0.00
4,210055092062,5,F,1,3,1,0,B,D,C,B,500.00,467.00,517.10,483.60,640.00


In [3]:
data_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44572 entries, 0 to 44571
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NU_INSCRICAO     44572 non-null  int64  
 1   TP_FAIXA_ETARIA  44572 non-null  int64  
 2   TP_SEXO          44572 non-null  object 
 3   TP_ESTADO_CIVIL  44572 non-null  int64  
 4   TP_COR_RACA      44572 non-null  int64  
 5   TP_ESCOLA        44572 non-null  int64  
 6   IN_TREINEIRO     44572 non-null  int64  
 7   Q001             44572 non-null  object 
 8   Q002             44572 non-null  object 
 9   Q006             44572 non-null  object 
 10  Q025             44572 non-null  object 
 11  NU_NOTA_CN       44572 non-null  float64
 12  NU_NOTA_CH       44572 non-null  float64
 13  NU_NOTA_LC       44572 non-null  float64
 14  NU_NOTA_MT       44572 non-null  float64
 15  NU_NOTA_REDACAO  44572 non-null  float64
dtypes: float64(5), int64(6), object(5)
memory usage: 5.4+ MB


We will create a new variable for the final grade of students that want to study at UnB based in the edital of 2022. Obs: the final grade is calculated differently for students that want to enroll in an exact sciences course or enroll in a humanities course. We choose to calculate the grade for exact science course.

In [4]:
data_DF['FINAL_GRADE']=(4*data_DF['NU_NOTA_CN'] + 4*data_DF['NU_NOTA_MT'] + 2*data_DF['NU_NOTA_CH'] + 2*data_DF['NU_NOTA_LC'] + data_DF['NU_NOTA_REDACAO'])/13
data_DF.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,Q001,Q002,Q006,Q025,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,FINAL_GRADE
0,210055243739,4,F,1,1,2,0,D,D,C,B,390.10,537.70,442.70,441.40,600.00,452.83
1,210055092856,2,F,1,3,2,0,D,E,E,B,564.00,581.30,604.50,590.40,780.00,597.63
2,210054977951,5,F,1,2,1,0,B,B,C,B,478.30,410.30,445.60,469.00,340.00,449.31
3,210055094382,2,F,1,1,1,1,C,D,B,B,483.00,487.30,501.40,540.00,0.00,466.88
4,210055092062,5,F,1,3,1,0,B,D,C,B,500.00,467.00,517.10,483.60,640.00,503.28


Now that we have a final grade we will try to predict it. We don't want the other grades influencing this predictions, then we drop those colums.

In [5]:
data_DF.drop(['NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_CH', 'NU_NOTA_LC','NU_NOTA_REDACAO'], axis=1, inplace=True)
data_DF.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,Q001,Q002,Q006,Q025,FINAL_GRADE
0,210055243739,4,F,1,1,2,0,D,D,C,B,452.83
1,210055092856,2,F,1,3,2,0,D,E,E,B,597.63
2,210054977951,5,F,1,2,1,0,B,B,C,B,449.31
3,210055094382,2,F,1,1,1,1,C,D,B,B,466.88
4,210055092062,5,F,1,3,1,0,B,D,C,B,503.28


As we can see in the pairplot grafics there is no clear correlation between the diferent variables we choose.

In [6]:
#sns.pairplot(data=data_DF)

## Codificação de variáveis categoricas ordinais:

In the DataFrame we have ordinal categorical variables that are not labeled with numbers, althoug machine learning models just work with numbers. Now we solve this problem by replacing the letters with numbers.

In [7]:
data_DF.Q001=data_DF.Q001.map( {'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':0})

data_DF.Q002=data_DF.Q002.map({'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':0})

data_DF.Q006=data_DF.Q006.map({'A': 1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11,
                               'L':12, 'M':13, 'N':14, 'O':15, 'P':16, 'Q':17})

## Codificação de variáveis categoricas nominais:

Some of the nominal categorical variables are also represented with letters, we solve this problem by replacing the letters with numbers.

In [ ]:
data_DF.TP_SEXO=data_DF.TP_SEXO.map({'F':1, 'M':0})

data_DF.Q025=data_DF.Q025.map({'A':0, 'B':1})

The columns TP_ESTADO_CIVIL and TP_COR_RACA are nominal categorical variables, they are alread labeled with numbers. Althoug we need to criate dummie variables to separate the categories.

In [8]:
columnTP_ESTADO_CIVIL =pd.get_dummies(data_DF['TP_ESTADO_CIVIL'], drop_first=True, dtype=int)

columnTP_ESTADO_CIVIL.rename(columns={1: 'TP_ESTADO_CIVIL_1', 2: 'TP_ESTADO_CIVIL_2', 3: 'TP_ESTADO_CIVIL_3', 4: 'TP_ESTADO_CIVIL_4'}, inplace=True)

columnTP_ESTADO_CIVIL.head()

,TP_ESTADO_CIVIL_1,TP_ESTADO_CIVIL_2,TP_ESTADO_CIVIL_3,TP_ESTADO_CIVIL_4
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [9]:
columnTP_ESTADO_CIVIL.columns

Index(['TP_ESTADO_CIVIL_1', 'TP_ESTADO_CIVIL_2', 'TP_ESTADO_CIVIL_3',
       'TP_ESTADO_CIVIL_4'],
      dtype='object')

In [10]:
columnTP_COR_RACA =pd.get_dummies(data_DF['TP_COR_RACA'], drop_first=True, dtype=int)

columnTP_COR_RACA.rename(columns={0: 'TP_COR_RACA_0', 1: 'TP_COR_RACA_1', 2: 'TP_COR_RACA_2', 3: 'TP_COR_RACA_3', 4: 'TP_COR_RACA_4', 5:'TP_COR_RACA_5', 6:'TP_COR_RACA_6' }, inplace=True)

columnTP_COR_RACA.head()

,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5
0,1,0,0,0,0
1,0,0,1,0,0
2,0,1,0,0,0
3,1,0,0,0,0
4,0,0,1,0,0


# Definindo X e y

Now it's time to separate our data into X and y. In this step we separate the colluns that don't belong to X in our data_DF and join with columnTP_ESTADO_CIVIL and columnTP_COR_RACA that we criate.

In [19]:
X = pd.concat([data_DF.drop(['NU_INSCRICAO', 'FINAL_GRADE', 'TP_ESTADO_CIVIL', 'TP_COR_RACA'], axis=1), columnTP_ESTADO_CIVIL, columnTP_COR_RACA], axis=1)
X.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESCOLA,IN_TREINEIRO,Q001,Q002,Q006,Q025,TP_ESTADO_CIVIL_1,TP_ESTADO_CIVIL_2,TP_ESTADO_CIVIL_3,TP_ESTADO_CIVIL_4,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5
0,4,1,2,0,4,4,3,1,1,0,0,0,1,0,0,0,0
1,2,1,2,0,4,5,5,1,1,0,0,0,0,0,1,0,0
2,5,1,1,0,2,2,3,1,1,0,0,0,0,1,0,0,0
3,2,1,1,1,3,4,2,1,1,0,0,0,1,0,0,0,0
4,5,1,1,0,2,4,3,1,1,0,0,0,0,0,1,0,0


The colum we want to predict is our FINAL_GRADE, then we take this column from our dataframe.

In [20]:
y=data_DF.FINAL_GRADE
y.head()

0   452.83
1   597.63
2   449.31
3   466.88
4   503.28
Name: FINAL_GRADE, dtype: float64

# Separação de bases

Now we need to split our data into a train set (were we are going to fit our model) and a test set (were we re going to make predictions and check with the actual value).

In [14]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size = .2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_train.shape

((35657, 17), (8915, 17), (35657,), (35657,))

# Feature Scaling

Our columns have very different scales, that is not apropriate for some machine learning models to work. Than, in this step we will normalize our data.

In [21]:
scaler=MinMaxScaler()

colunas = X_train.columns

X_train[colunas] = scaler.fit_transform(X_train)
X_test[colunas] = scaler.transform(X_test)

In [22]:
X_train.describe()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESCOLA,IN_TREINEIRO,Q001,Q002,Q006,Q025,TP_ESTADO_CIVIL_1,TP_ESTADO_CIVIL_2,TP_ESTADO_CIVIL_3,TP_ESTADO_CIVIL_4,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5
count,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00,35657.00
mean,0.18,0.61,0.28,0.13,0.63,0.71,0.36,0.96,0.92,0.04,0.02,0.00,0.43,0.12,0.41,0.02,0.00
std,0.18,0.49,0.35,0.33,0.30,0.24,0.30,0.19,0.28,0.19,0.12,0.03,0.49,0.33,0.49,0.14,0.05
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.05,0.00,0.00,0.00,0.43,0.57,0.12,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.11,1.00,0.00,0.00,0.71,0.71,0.25,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.21,1.00,0.50,0.00,0.86,0.86,0.56,1.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [23]:
X_test.describe()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESCOLA,IN_TREINEIRO,Q001,Q002,Q006,Q025,TP_ESTADO_CIVIL_1,TP_ESTADO_CIVIL_2,TP_ESTADO_CIVIL_3,TP_ESTADO_CIVIL_4,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5
count,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00,8915.00
mean,0.18,0.61,0.28,0.13,0.63,0.72,0.37,0.96,0.92,0.04,0.02,0.00,0.43,0.12,0.40,0.02,0.00
std,0.18,0.49,0.36,0.33,0.30,0.24,0.30,0.19,0.27,0.18,0.12,0.02,0.50,0.32,0.49,0.13,0.05
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.05,0.00,0.00,0.00,0.43,0.57,0.12,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.11,1.00,0.00,0.00,0.71,0.71,0.25,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.21,1.00,0.50,0.00,0.86,0.86,0.56,1.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


# Exporting the data

Finally its time to export our data to be used in the machine learning models.

In [18]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv',index = False)
y_test.to_csv('y_test.csv',index = False)